# All Ireland M.bovis strain diversity and clade identification

In [1]:
import sys,os,io,shutil,subprocess
import glob
from importlib import reload
import numpy as np
import pandas as pd
pd.set_option('display.width', 200)
import pylab as plt
import matplotlib as mpl
from Bio import SeqIO
import seaborn as sns
sys.path.append('pathogenie')
from snipgenie import tools, aligners, app, trees, plotting
import toytree, toyplot
import toyplot.pdf
path = '/storage/btbgenie/all_ireland_results'

## get meta data

In [2]:
meta = pd.read_csv('/storage/btbgenie/mbovis_ireland/all_ireland_samples.csv')
print (len(meta))

583


In [ ]:
dups=meta[meta.duplicated('aliquot')].dropna()
#print (dups)
for i,r in dups.iterrows():
    print (meta[meta.aliquot== r.aliquot])
#meta[(meta.batch.isin([2,3,4])) & (~meta.aliquot.isin(dups.aliquot)) & (meta.county=='Monaghan')]

## run NI

In [ ]:
reload(app)
args = {'threads':20, 'outdir': '/storage/btbgenie/NI_results', 'labelsep':'_',
        'input':['/storage/btbgenie/mbovis_ireland/NI'],
         'species': 'Mbovis-AF212297',
         'overwrite':False,
         'custom_filters': True,       
         'buildtree':True}
W = app.WorkFlow(**args)
st = W.setup()
W.run()

## misc 2021 samples

In [ ]:
args = {'threads':20, 'outdir': '/storage/btbgenie/misc_results', 'labelsep':'_',
        'input':['/storage/btbgenie/mbovis_ireland/Misc'],
         'species': 'Mbovis-AF212297',
         'overwrite':False,
         'custom_filters': True,
         'buildtree':True}
W = app.WorkFlow(**args)
st = W.setup()
W.run()

In [ ]:
reload(app)
mask='/home/farrell/gitprojects/snipgenie/snipgenie/data/Mbovis_AF212297_mask.bed'
vcf_file='/storage/btbgenie/misc_results/snps.vcf.gz'
app.mask_filter(vcf_file, mask)

In [244]:
present=pd.read_csv('/storage/btbgenie/misc_results/summary.csv')
present['sample']=present['sample'].astype(str)
#print (present[:10])
l=list(W.fastq_table['sample'].unique())

s=meta[meta.batch>=3].sort_values(by='sample')
#s['label'] = s['sample'].apply(lambda x : x.split('-')[1][-4:]).astype(str)
#s=s.sort_values('label')

p = present[~present['sample'].isin(s['sample'])]
#p = (s[s['label'].isin(l)])
print (p)
print (len(s),len(p), len(present))

         sample                           name                                           bam_file  read_length
25  TB19-004280  TB19-004280_S68_L001-4_R2_001  /storage/btbgenie/misc_results/mapped/TB19-004...          149
48  TB19-005887  TB19-005887_S29_L001-4_R2_001  /storage/btbgenie/misc_results/mapped/TB19-005...          150
65 2 61


In [284]:
reload(trees)
spo=pd.read_csv('/storage/btbgenie/all_ireland_results/misc_spoligotypes.csv')
treefile = '/storage/btbgenie/misc_results/tree.newick'
tre = toytree.tree(treefile)
tipnames = tre.get_tip_labels()
mapping = dict(zip(meta['sample'],meta.county))
colormap = trees.colors_from_labels(meta,'name','county')
tip_colors = [colormap[mapping[i]] if i in mapping else 'Black' for i in tipnames]

canvas,t,r=tre.draw(layout='r',width=1000,height=1000,tip_labels_colors=tip_colors,
                    scalebar=True,);
toyplot.pdf.render(canvas,'/storage/btbgenie/all_ireland_results/misc_samples_tree.pdf')

## run CVRL samples

In [ ]:
args = {'threads':20, 'outdir': '/storage/btbgenie/cvrl_results', 'labelsep':'_',
        'input':['/storage/btbgenie/mbovis_ireland/CVRL'],
         'species': 'Mbovis-AF212297',
         'overwrite':False,
         'custom_filters': True,
         'buildtree':True}
W = app.WorkFlow(**args)
st = W.setup()
W.run()

In [ ]:
reload(trees)
spo=pd.read_csv('/storage/btbgenie/all_ireland_results/cvrl_spoligotypes.csv')
treefile = '/storage/btbgenie/cvrl_results/tree.newick'
tre = toytree.tree(treefile)
tipnames = tre.get_tip_labels()
mapping = dict(zip(meta['sample'],meta.county))
colormap = trees.colors_from_labels(meta,'name','county')
tip_colors = [colormap[mapping[i]] if i in mapping else 'Black' for i in tipnames]

canvas,t,r=tre.draw(layout='r',width=1000,height=1400,tip_labels_colors=tip_colors,
                    scalebar=True,);

## integrate previously aligned bam files together and call variants

This allows us to join previously mapped results together in one call without re-aligning

In [4]:
#read in bam files
mapped = {'wicklow':'/storage/btbgenie/wicklow_results/mapped/',
          'cvrl':'/storage/btbgenie/cvrl_results/mapped/',
          'misc':'/storage/btbgenie/misc_results/mapped/',
          'monaghan':'/storage/btbgenie/monaghan/monaghan_results/mapped/',
          'NI':'/storage/btbgenie/NI_results/','UK':'/storage/btbgenie/UK_results/mapped'}
reload(app)
bam_files=[]
for m in mapped:
    f=app.get_files_from_paths(mapped[m], '*.bam')
    bam_files.extend(f)
print (len(bam_files))

outdir='/storage/btbgenie/all_ireland_results'
df = app.get_samples(bam_files,sep='_')
app.write_samples(df, outdir)

531


'/storage/btbgenie/all_ireland_results/samples.txt'

## run previously aligned files

In [5]:
gff_file=os.path.join(outdir,'mbovis.gff')
tools.gff_bcftools_format(app.mbovis_gb, gff_file)
app.run_bamfiles(bam_files, app.mbovis_genome, outdir='/storage/btbgenie/all_ireland_results', threads=20,
                 mask=app.mbovis_mask, gff_file=gff_file,
                 custom_filters=True, overwrite=False)

/storage/btbgenie/all_ireland_results/raw.bcf already exists
calling variants..
bcftools call --ploidy 1 -m -v -o /storage/btbgenie/all_ireland_results/calls.vcf /storage/btbgenie/all_ireland_results/raw.bcf
/storage/btbgenie/all_ireland_results/samples.txt
bcftools reheader --samples /storage/btbgenie/all_ireland_results/samples.txt -o /tmp/calls.vcf /storage/btbgenie/all_ireland_results/calls.vcf
bcftools filter -i "QUAL>=40 && FORMAT/DP>=30 && DP4>=4" -o /storage/btbgenie/all_ireland_results/filtered.vcf.gz -O z /storage/btbgenie/all_ireland_results/calls.vcf
splitting snps and indels..
bcftools view -v snps -o /storage/btbgenie/all_ireland_results/snps.vcf.gz -O z /storage/btbgenie/all_ireland_results/filtered.vcf.gz
bcftools view -v indels -o /storage/btbgenie/all_ireland_results/indels.vcf.gz -O z /storage/btbgenie/all_ireland_results/filtered.vcf.gz
using mask bed file /home/farrell/gitprojects/snipgenie/snipgenie/data/Mbovis_AF212297_mask.bed
6502 sites
found 812 sites in maske

## spoligotypes of all samples

In [3]:
paths = {'Wicklow':'/storage/btbgenie/mbovis_ireland/Wicklow','Monaghan':'/storage/btbgenie/mbovis_ireland/Monaghan/',
         'cvrl':'/storage/btbgenie/mbovis_ireland/CVRL',
         'NI':'/storage/btbgenie/mbovis_ireland/NI/'}
samples = []
#paths = {'misc':'/storage/btbgenie/mbovis_ireland/Jun21/'}
for p in paths:
    f = app.get_files_from_paths(paths[p])   
    df = app.get_samples(f,sep='_')
    print (len(df['sample'].unique()))
    df=df.drop_duplicates('sample')   
    df['county'] = p   
    samples.append(df)

samples = pd.concat(samples)

47
175
205
31


In [ ]:
reload(tools)
res=[]
for i,r in samples.iterrows():
    f=r.filename
    samp=r['sample']
    b = tools.get_spoligotype(f,1e6)
    sb = tools.get_sb_number(b)
    print (f,samp,sb, b)
    res.append([f,samp,sb,b])
    
res = pd.DataFrame(res,columns=['filename','sample','SB','code'])
res.code.astype(object)
#res.to_csv('/storage/btbgenie/all_ireland_results/spoligotypes.csv',index=False)

In [215]:
#res.to_csv('/storage/btbgenie/all_ireland_results/misc_spoligotypes.csv',index=False)

## clusters

In [6]:
clust = trees.run_tree_cluster('/storage/btbgenie/all_ireland_results/tree.newick',dist=10)
#clust = pd.read_csv('monaghan_results/clusters.txt',sep='\t')
clust.ClusterNumber.value_counts()


TreeCluster.py  -i /storage/btbgenie/all_ireland_results/tree.newick -t 10


-1     154
 29     39
 48     25
 4      20
 43     18
      ... 
 56      2
 8       2
 58      2
 59      2
 50      2
Name: ClusterNumber, Length: 83, dtype: int64

In [ ]:
meta.county.value_counts()

In [9]:
spo = pd.read_csv('/storage/btbgenie/all_ireland_results/spoligotypes.csv')
#clust = pd.read_csv('/storage/btbgenie/all_ireland_results/clusters.txt',sep='\t')

In [10]:
final = meta.merge(spo,on='sample',how='left')
final = final.drop('code',1)
final = final.merge(clust,left_on='sample',right_on='SequenceName')
len(final)

521

## assign clade names?

In [11]:
clmap={}
for county,g in final.groupby('county'):
    c = g.value_counts('ClusterNumber')
    c = c[c>8]
    n=1
    for i,r in c.iteritems():
        if i==-1:
            continue
        clmap[i]=county+'-'+str(n)
        n+=1
clmap

{48: 'Clare-1', 4: 'Clare-2', 43: 'Clare-3', 29: 'Monaghan-1'}

In [12]:
final['clade'] = final.ClusterNumber.map(clmap)

In [13]:
final.to_csv('/storage/btbgenie/mbovis_ireland/ireland_metadata.csv',index=False)

## draw all ireland tree

In [14]:
treefile = '/storage/btbgenie/all_ireland_results/tree.newick'
tre = toytree.tree(treefile)
#canvas,t,r=tre.draw(layout='c')

In [21]:
final[:20]

,sample,county,species,source,batch,aliquot,filename,SB,SequenceName,ClusterNumber,clade,color
0,ERR125617,NI,Cow,Other,NaN,NaN,/storage/btbgenie/mbovis_ireland/NI/ERR125617_...,SB0140,ERR125617,-1,NaN,blue
1,ERR125622,NI,Cow,Other,NaN,NaN,/storage/btbgenie/mbovis_ireland/NI/ERR125622_...,SB0140,ERR125622,16,NaN,blue
2,ERR125598,NI,Cow,Other,NaN,NaN,/storage/btbgenie/mbovis_ireland/NI/ERR125598_...,SB0140,ERR125598,-1,NaN,blue
3,ERR125612,NI,Cow,Other,NaN,NaN,/storage/btbgenie/mbovis_ireland/NI/ERR125612_...,SB0140,ERR125612,69,NaN,blue
4,ERR125603,NI,Cow,Other,NaN,NaN,/storage/btbgenie/mbovis_ireland/NI/ERR125603_...,SB0140,ERR125603,43,Clare-3,blue
5,ERR125611,NI,Cow,Other,NaN,NaN,/storage/btbgenie/mbovis_ireland/NI/ERR125611_...,SB0140,ERR125611,69,NaN,blue
6,ERR125600,NI,Cow,Other,NaN,NaN,/storage/btbgenie/mbovis_ireland/NI/ERR125600_...,SB0140,ERR125600,23,NaN,blue
7,ERR125626,NI,Cow,Other,NaN,NaN,/storage/btbgenie/mbovis_ireland/NI/ERR125626_...,SB0140,ERR125626,42,NaN,blue
8,ERR125625,NI,Cow,Other,NaN,NaN,/storage/btbgenie/mbovis_ireland/NI/ERR125625_...,SB0140,ERR125625,-1,NaN,blue
9,ERR125616,NI,Cow,Other,NaN,NaN,/storage/btbgenie/mbovis_ireland/NI/ERR125616_...,SB0140,ERR125616,24,NaN,blue


In [19]:
reload(trees)
#tre = tre.root('TB20-002800')
#tre = tre.drop_tips('8567')
tipnames = tre.get_tip_labels()
mapping = dict(zip(final['sample'],final.SB))
#mapping['ref'] = 'AF2122/97'
tiplabels = [mapping[i] if i in mapping else 'NA' for i in tipnames]

colormap = trees.colors_from_labels(final,'name','SB')
print (colormap)
tip_colors = [colormap[mapping[i]] if i in mapping else 'Black' for i in tipnames]
tip_labels_style={      
        "font-size": "6px",
        "-toyplot-anchor-shift": "13px",
    }
node_sizes=[0 if i else 5 for i in tre.get_node_values(None, 1, 0)]
node_colors = [colormap[mapping[n]] if n in mapping else 'black' for n in tre.get_node_values('name', True, True)]

canvas,t,r=tre.draw(layout='r',width=1200,height=1300,node_markers="o",node_hover=True, node_sizes=node_sizes,
                    tip_labels=False,tip_labels_style=tip_labels_style,node_colors=node_colors,
                    tip_labels_colors=tip_colors,scalebar=True,edge_widths=1);
#toyplot.pdf.render(canvas,'/storage/btbgenie/all_ireland_results/ireland_tree.pdf')

{'SB0140': 'blue', 'SB0054': 'green', 'SB0142': 'crimson', nan: 'blueviolet', 'SB0975': 'orange', 'SB0263': 'cadetblue', 'SB0980': 'chartreuse', 'SB0145': 'chocolate', 'SB0146': 'coral', 'SB0978': 'gold', 'SB0129': 'cornflowerblue', 'SB2054': 'palegreen', 'SB0130': 'khaki', 'SB1036': 'orange', 'SB0344': 'pink', 'SB0673': 'burlywood', 'SB0914': 'red', 'SB1066': 'lime', 'SB0666': 'mediumvioletred', 'SB1524': 'navy', 'SB1762': 'teal', 'SB2487': 'darkblue', 'SB0272': 'purple', 'SB2255': 'orange', 'SB0139': 'salmon', 'SB0141': 'maroon', 'SB2565': 'blue', 'SB2504': 'green', 'SB1184': 'crimson', 'SB1531': 'blueviolet', 'SB1646': 'orange', 'SB1049': 'cadetblue', 'SB1382': 'chartreuse', 'SB1319': 'chocolate', 'SB1924': 'coral', 'SB1033': 'gold', 'SB2085': 'cornflowerblue', 'SB2627': 'palegreen', 'SB1348': 'khaki', 'SB2486': 'orange', 'SB0274': 'pink', 'SB0275': 'burlywood', 'SB1789': 'red', 'SB0993': 'lime', 'SB0273': 'mediumvioletred', 'SB2295': 'navy', 'SB1072': 'teal', 'SB0298': 'darkblue'}


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1200.0px" height="1300.0px" viewBox="0 0 1200.0 1300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t79400db7b13f4fdb994bd40b0ede0d90"> idx: 0
dist: 0.0049
support: 0.0000
height: 303.6503
name: TB20-005617 idx: 1
dist: 0.0049
support: 0.0000
height: 303.6503
name: TB20-001128 idx: 2
dist: 0.0049
support: 0.0000
height: 302.6549
name: TB20-001041 idx: 3
dist: 2.0013
support: 0.0000
height: 300.6585
name: TB19-009502 idx: 4
dist: 0.0049
support: 0.0000
height: 303.6554
name: 52-3698 idx: 5
dist: 0.0049
support: 0.0000
height: 303.6554
name: TB20-001937 idx: 6
dist: 16.0392
support: 0.0000
height: 287.6111
name: TB19-008302 idx: 7
dist: 1.0003
support: 0.0000
height: 302.6500
name: TB20-003627 idx: 8
dist: 0.0049
support: 0.0000
height: 303.6503
name: TB19-005742 idx: 9
dist: 1.0004
support: 0.0000
height: 303.6552
name: TB19-007948 idx: 10
dist: 0.0049
support: 0.0000
height: 304.6555
name: TB19-005728 idx: 11
dist: 0.0049
support: 0.0000
height: 304.6604
name: TB19-006652 idx: 12
dist: 0.0049
support: 0.0000
height: 304.6604
name: TB19-008458 idx: 13
dist: 0.0049
support: 0.0000
height: 303.6454
name: TB19-007878 idx: 14
dist: 0.0049
support: 0.0000
height: 303.6454
name: TB19-011417 idx: 15
dist: 0.0049
support: 0.0000
height: 303.6503
name: TB20-001436 idx: 16
dist: 0.0049
support: 0.0000
height: 303.6552
name: TB20-001461 idx: 17
dist: 0.0049
support: 0.0000
height: 303.6601
name: TB20-002906 idx: 18
dist: 1.0003
support: 0.0000
height: 302.6696
name: TB19-005738 idx: 19
dist: 1.0003
support: 0.0000
height: 303.6700
name: TB19-009477 idx: 20
dist: 1.0002
support: 0.0000
height: 303.6798
name: TB19-008474 idx: 21
dist: 0.0049
support: 0.0000
height: 304.6602
name: TB19-011599 idx: 22
dist: 0.0049
support: 0.0000
height: 304.6602
name: TB20-001911 idx: 23
dist: 0.0049
support: 0.0000
height: 303.6646
name: TB19-006656 idx: 24
dist: 2.0013
support: 0.0000
height: 301.6682
name: TB19-009474 idx: 25
dist: 0.0049
support: 0.0000
height: 305.6609
name: TB20-002800 idx: 26
dist: 0.0049
support: 0.0000
height: 305.6609
name: TB19-005887 idx: 27
dist: 1.0005
support: 0.0000
height: 304.6701
name: TB19-007739 idx: 28
dist: 0.0049
support: 0.0000
height: 305.6657
name: TB19-006577 idx: 29
dist: 3.0022
support: 0.0000
height: 301.6631
name: TB19-009558 idx: 30
dist: 1.0004
support: 0.0000
height: 303.6649
name: 19-7517 idx: 31
dist: 0.0049
support: 0.0000
height: 304.6653
name: 2975 idx: 32
dist: 0.0049
support: 0.0000
height: 305.6658
name: TB20-001642 idx: 33
dist: 1.0004
support: 0.0000
height: 305.6707
name: TB20-000241 idx: 34
dist: 0.0049
support: 0.0000
height: 310.6757
name: TB19-000499 idx: 35
dist: 0.0049
support: 0.0000
height: 310.6757
name: ERR125614 idx: 36
dist: 1.0006
support: 0.0000
height: 309.6849
name: 356 idx: 37
dist: 1.0009
support: 0.0000
height: 309.6895
name: 2114p idx: 38
dist: 1.0010
support: 0.0000
height: 309.6894
name: TB20-002585 idx: 39
dist: 3.0022
support: 0.0000
height: 308.6935
name: 48-MBovis idx: 40
dist: 8.0165
support: 0.0000
height: 305.6812
name: TB19-011426 idx: 41
dist: 7.1207
support: 0.0000
height: 304.4663
name: TB19-003842 idx: 42
dist: 4.8987
support: 0.0000
height: 306.6883
name: TB19-006259 idx: 43
dist: 6.9032
support: 0.0000
height: 305.6848
name: TB20-001910 idx: 44
dist: 8.0156
support: 0.0000
height: 305.6842
name: 3496 idx: 45
dist: 0.0049
support: 0.0000
height: 305.6681
name: TB20-002811 idx: 46
dist: 1.0005
support: 0.0000
height: 304.6726
name: TB20-003043 idx: 47
dist: 1.0005
support: 0.0000
height: 304.6775
name: TB19-008697 idx: 48
dist: 1.0005
support: 0.0000
heigh

## SNP analysis

* snps unique to each clade/county
* protein groups containing common snps

In [35]:
csq = app.read_csq_file('/storage/btbgenie/all_ireland_results/csq.tsv')
snp_mat = app.get_aa_snp_matrix(csq)
#csq_ind = app.read_csq_file('/storage/btbgenie/all_ireland_results/csq_indels.tsv')
#ind_mat = app.get_aa_snp_matrix(csq_ind)

In [36]:
mat = snp_mat

#labelmap = dict(zip(meta['name'],meta.county))
#mat = mat.rename(columns=labelmap)
#freqs = mat.sum(1)/len(mat.columns)
#freqs =freqs.rename('freq') 
#freqs.sort_values()
#freqs.hist(bins=20)

In [37]:
#mat.index = mat.index.get_level_values(1)+'_'+mat.index.get_level_values(2)
stat = mat.sum(1).reset_index()
#stat.gene.value_counts()[:30]

In [ ]:
cols=['sample','county','SB','ClusterNumber','clade']
C=mat.T
C=C.merge(meta[cols],left_index=True,right_on='sample').set_index(cols).T
#print (C)
C=C[C.sum(1)>3].T
C.iloc[:4,:2]
C.to_csv('csq_snps_ireland.csv')

In [257]:
counties = list(C.index.get_level_values(1))
sb = list(C.index.get_level_values(2))
clust = C.index.get_level_values(3)
clades = C.index.get_level_values(4)

In [ ]:
colormap1 = dict(zip(clades.unique(), trees.qcolors))
#colormap1 = {'Wicklow-1': 'green', 'Monaghan-3': 'red', np.nan: 'gray', 
#             'Monaghan-1': 'brown', 'Monaghan-2': 'burlywood', 'Unknown': 'lightblue', 'NI-1': 'blue'}
colors1 = [colormap1[i] if i in colormap1 else 'Black' for i in clades]
#make legend
import matplotlib.patches as mpatches
pts=[]
for c in colormap1:
    pts.append(mpatches.Patch(color=colormap1[c],label=c))
g=sns.clustermap(C,xticklabels=False,yticklabels=False,figsize=(17,12),cmap='gray_r',cbar_pos=None,row_colors=colors1)
g.fig.legend(handles=pts,bbox_to_anchor=(1.05, 1), )
g.savefig('/storage/btbgenie/all_ireland_results/snp_clusters.jpg')

## PCA

In [33]:
import sklearn
from sklearn import decomposition
from sklearn import manifold

pca = decomposition.PCA(n_components=3)
mds = manifold.MDS(n_components=3)
pos = mds.fit(C).embedding_
X = pca.fit_transform(pos)
#X = pca.transform(C)
df=pd.DataFrame(X)
df['sb']=sb
df.to_csv('/storage/btbgenie/all_ireland_results/pca_snp.csv')

fig,ax = plt.subplots(figsize=(7, 7))
sc=ax.scatter(X[:, 0], X[:, 1], c=colors1, s=100, alpha=.6)    
ax.legend(handles=pts,loc='best')
fig.savefig('/storage/btbgenie/all_ireland_results/pca_snp_clusters.jpg',dpi=150)

NameError: name 'C' is not defined

## compare cvrl wgmlst tree

In [351]:
cvrltre = toytree.tree('declan.newick')
cvrltre.draw(layout='c',width=800,height=1000,
                node_markers="o",node_hover=True, #node_sizes=node_sizes,
                tip_labels=False,tip_labels_style=tip_labels_style,#node_colors=node_colors,
                scalebar=True,edge_widths=1)

(<toyplot.canvas.Canvas at 0x7f1032dd6d00>,
 <toytree.Render.ToytreeMark at 0x7f1032389550>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1000.0px" height="1000.0px" viewBox="0 0 1000.0 1000.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t1ac6205ac4f243e3a41c08510318b498"> -200 -100 0 100 200 -200 -100 0 100 200